In [14]:
import os
import openai
import pyodbc
from dotenv import load_dotenv
import pandas as pd

def load_api_key():
    """Load the OpenAI API key from an environment variable."""
    load_dotenv()  # Load environment variables from a .env file (optional)
    return os.getenv("OPENAI_API_KEY")  # Make sure the key is set in your environment

def get_user_query():
    """Prompt the user or read from input."""
    return input("Give me a list of brands: ")

def generate_sql_query(natural_language_query, schema_info=""):
    """
    Send the user query (and optional schema info) to the OpenAI API
    and return the generated SQL query.
    """
    # Craft a system prompt or instructions:
    system_prompt = f"""
    You are an expert SQL assistant. Given the database schema:
    {schema_info}

    Convert the following natural language request into a SQL query:
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": natural_language_query}
        ],
        max_tokens=150,
        temperature=0.0,  # Keep it deterministic for predictable SQL
    )

    # Extract the generated text (SQL query) from the response
    sql_query = response.choices[0].message["content"]
    return sql_query.strip()

def execute_query(sql_query):
    """Execute the generated SQL query on a SQL Server instance running in Docker."""
    try:
        # Update server=localhost,1433 -- no space between host and port
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=localhost,1433;"
            "DATABASE=SampleDB;"
            "UID=sa;"
            "PWD=dockerStrongPwd123;"
        )
        cursor = conn.cursor()

        cursor.execute(sql_query)
        if cursor.description:  # If it's a SELECT query, fetch results
            records = cursor.fetchall()
            # Print or process each row in the result
            for row in records:
                print(row)
        else:
            # For INSERT, UPDATE, etc., commit changes
            conn.commit()
            print("Query executed successfully!")
    except Exception as e:
        print("Error executing query:", e)
    finally:
        cursor.close()
        conn.close()

def main():
    # 1. Load API key
    openai_api_key = load_api_key()
    if not openai_api_key:
        print("Error: OpenAI API key not found.")
        return
    openai.api_key = openai_api_key

    # 2. Get user input
    user_query = get_user_query()

    # 3. (Optional) Provide schema info (hardcoded example)
    schema_info = """
    Table:
      - BrandData(brand VARCHAR(50), count VARCHAR(255), percentage FLOAT, state VARCHAR(50), year VARCHAR(255), quater INT)
    """

    # 4. Generate the SQL query from the user’s request
    print("Generating SQL from natural language...")
    sql_query = generate_sql_query(user_query, schema_info)
    print(f"Generated SQL query:\n{sql_query}\n")

    # 5. Execute the SQL query
    print("Executing query...")
    execute_query(sql_query)

if __name__ == "__main__":
    main()


Generating SQL from natural language...


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [12]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost,1433;"
    "DATABASE=master;"        # or any DB you want to connect to
    "UID=sa;"
    "PWD=dockerStrongPwd123;"
)
print("Connection successful!")
conn.close()

Connection successful!
